# Fine-tuning the CNN and concept models

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from torch import save
import torch
import csv
import sys
from torchinfo import summary

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import seaborn as sns
sns.set_theme(style="darkgrid")

In [2]:
%load_ext autoreload
%autoreload 2
from LIDCDataModule import LIDCDataModule
from CNNModelFinetune import CNNModelFinetune
from conceptModelFinetune import conceptModelFinetune

In [3]:
device = "cuda:0"

In [10]:
data_path = "/home/lbrocki/ConRad/dataset"
weights_path = "/home/lbrocki/ConRad/weights"

### fine-tuning CNN model

In [ ]:
# train on 5 folds
for fold in range(5):
    print(f"fold: {fold}")
    trainer = Trainer(
        gpus=[0],
        accelerator="gpu",
        max_epochs=50,
        callbacks=[
            LearningRateMonitor(logging_interval="epoch"), 
            TQDMProgressBar(refresh_rate=10),
        ],
        precision=16
    )
    model = CNNModelFinetune(learning_rate=1e-3)
    data_module = LIDCDataModule(
        data_dir=data_path, 
        fold=fold, 
        batch_size=32, 
        labels="targets",
        finetune=True
    )
    trainer.fit(model, data_module)
    torch.save(model.state_dict(), f"{weights_path}/cnn_finetune_{fold}.pt")

### fine-tuning concept model

In [ ]:
# train on 5 folds
for fold in range(5):
    print(f"fold: {fold}")
    trainer = Trainer(
        gpus=[0],
        accelerator="gpu",
        max_epochs=50,
        callbacks=[
            LearningRateMonitor(logging_interval="epoch"), 
            TQDMProgressBar(refresh_rate=10),
        ],
        precision=16
    )
    model = conceptModelFinetune(learning_rate=1e-3)
    data_module = LIDCDataModule(
        data_dir=data_path, 
        fold=fold, 
        batch_size=32, 
        labels="concepts",
        finetune=True
    )
    trainer.fit(model, data_module)
    torch.save(model.state_dict(), f"{weights_path}/concept_finetune_{fold}.pt")

### evaluating performance of CNN model

In [ ]:
trainer = Trainer(
    gpus=[0],
    accelerator="gpu",
    precision=16
)
acc = []
auroc = []
precision = []
recall = []
roc = []
for fold in range(5):
    print(fold)
    data_module = LIDCDataModule(data_dir="dataset_masks", fold=fold, finetune=True, num_workers=4)
    model = CNNModelFinetune()
    model.load_state_dict(torch.load(f"weights/cnn_finetune_25_01_{fold}.pt"))
    model.eval()
    res = trainer.test(model, data_module)
    acc.append(res[0]["val_acc"])
    auroc.append(res[0]["val_auroc"])
    precision.append(res[0]["val_precision"])
    recall.append(res[0]["val_recall"])
#     roc.append(res[0]["val_roc"])

In [41]:
np.mean(precision)

0.8854260563850402

In [43]:
np.mean(acc)

0.891104233264923

In [44]:
np.mean(recall)

0.8831798315048218